In [22]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import os
import wandb
from pathlib import Path

# 본 과제 제출자는 현재 우분투 도커 환경에서 작업중이므로 다음과 같이 경로 설정
BASE_PATH="/home/Deep-Learning-study"
import sys
sys.path.append(BASE_PATH)

CURRENT_FILE_PATH = os.getcwd()
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")

if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

In [23]:
from _01_code._15_lstm_and_its_application.f_arg_parser import get_parser
from _01_code._14_rnn.g_rnn_trainer import RegressionTrainer
from _01_code._03_real_world_data_to_tensors.p_cryptocurrency_dataset_dataloader import get_cryptocurrency_data, \
  CryptoCurrencyDataset

In [24]:
def get_btc_krw_data(sequence_size=10, validation_size=100, test_size=10, is_regression=True):
    # 비트코인 데이터 로드 및 전처리
    # X: 입력 특성 (OHLCV), y: 타겟값 (다음 날 종가), date: 날짜 정보
    X_train, X_validation, X_test, y_train, y_validation, y_test, y_train_date, y_validation_date, y_test_date \
        = get_cryptocurrency_data(
            sequence_size=sequence_size,        # LSTM에 입력될 시퀀스 길이
            validation_size=validation_size,    # 검증 데이터셋 크기
            test_size=test_size,               # 테스트 데이터셋 크기
            target_column='Close',             # 예측할 대상 컬럼 (종가)
            y_normalizer=1.0e7,                # 타겟값 정규화를 위한 스케일링 계수
            is_regression=is_regression         # True: 회귀, False: 분류
        )

    # PyTorch Dataset 객체 생성
    # 각 데이터셋을 PyTorch에서 사용할 수 있는 형태로 변환
    train_crypto_currency_dataset = CryptoCurrencyDataset(X=X_train, y=y_train)
    validation_crypto_currency_dataset = CryptoCurrencyDataset(X=X_validation, y=y_validation)
    test_crypto_currency_dataset = CryptoCurrencyDataset(X=X_test, y=y_test)

    # DataLoader 생성
    # 학습 데이터 로더: 미니배치 단위로 데이터를 제공
    train_data_loader = DataLoader(
        dataset=train_crypto_currency_dataset, 
        batch_size=wandb.config.batch_size,    # wandb에 설정된 배치 크기 사용
        shuffle=True                           # 에폭마다 데이터 순서를 섞음
    )
    
    # 검증 데이터 로더: 모델 성능 평가용
    validation_data_loader = DataLoader(
        dataset=validation_crypto_currency_dataset, 
        batch_size=wandb.config.batch_size,    # 학습과 동일한 배치 크기 사용
        shuffle=True                           # 검증 데이터도 섞어서 사용
    )
    
    # 테스트 데이터 로더: 최종 성능 평가용
    test_data_loader = DataLoader(
        dataset=test_crypto_currency_dataset,
        batch_size=len(test_crypto_currency_dataset),  # 테스트는 전체 데이터를 한 번에 처리
        shuffle=True                                   # 테스트 데이터도 섞어서 사용
    )

    return train_data_loader, validation_data_loader, test_data_loader

In [31]:
def get_model():
    class MyModel(nn.Module):
        def __init__(self, n_input, n_output):
            super().__init__()
            
            # 단일 LSTM으로 변경하고 hidden size 증가
            self.lstm = nn.LSTM(
                input_size=n_input,
                hidden_size=1024,  # hidden size 대폭 증가
                num_layers=3,      # layer 수 증가
                dropout=0.1,       # dropout 감소
                batch_first=True,
                bidirectional=True # 양방향 LSTM 사용
            )
            
            # FC 레이어 단순화
            self.fc_layers = nn.Sequential(
                nn.LayerNorm(2048),  # bidirectional이므로 hidden_size * 2
                nn.Linear(2048, 512),
                nn.GELU(),           # GELU 활성화 함수 사용
                nn.Dropout(0.1),
                
                nn.LayerNorm(512),
                nn.Linear(512, n_output)
            )

        def forward(self, x):
            self.lstm.flatten_parameters()  # CUDA 성능 최적화
            x, _ = self.lstm(x)
            x = x[:, -1, :]  # 마지막 시퀀스의 출력만 사용
            x = self.fc_layers(x)
            return x

    my_model = MyModel(n_input=5, n_output=1)
    return my_model

class Args:
    def __init__(self):
        self.wandb = True
        self.batch_size = 16       # 배치 사이즈 감소
        self.epochs = 500          # epoch 증가
        self.learning_rate = 5e-4  # 학습률 조정
        self.weight_decay = 1e-5   # weight decay 감소
        self.validation_intervals = 1
        self.early_stop_patience = 50  # patience 증가
        self.early_stop_delta = 1e-6   # delta 감소

In [32]:
def main(args):
    # 현재 시간을 문자열로 변환 (실행 식별자로 사용)
    run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # wandb 설정을 위한 하이퍼파라미터 딕셔너리 생성
    config = {
        'epochs': args.epochs,                    # 총 학습 에폭 수
        'batch_size': args.batch_size,           # 미니배치 크기
        'validation_intervals': args.validation_intervals,  # 검증 수행 주기
        'learning_rate': args.learning_rate,      # 학습률
        'early_stop_patience': args.early_stop_patience,  # 조기 종료 인내 횟수
        'early_stop_delta': args.early_stop_delta,  # 조기 종료 임계값
        'weight_decay': args.weight_decay         # L2 정규화 강도
    }

    # wandb 프로젝트 초기화
    project_name = "lstm_regression_btc_krw"
    wandb.init(
        mode="online" if args.wandb else "disabled",  # wandb 활성화 여부
        project=project_name,                         # 프로젝트 이름
        notes="btc_krw experiment with lstm",         # 실험 설명
        tags=["lstm", "regression", "btc_krw"],       # 관련 태그
        name=run_time_str,                           # 실행 이름 (시간 기반)
        config=config                                # 설정값들
    )
    # 설정값 출력
    print(args)
    print(wandb.config)

    # 데이터 로더 생성 (테스트 데이터 로더는 현재 사용하지 않음)
    train_data_loader, validation_data_loader, _ = get_btc_krw_data()
    
    # GPU 사용 가능 여부 확인 및 디바이스 설정
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Training on device {device}.")

    # 모델 인스턴스 생성 및 지정된 디바이스로 이동
    model = get_model()
    model.to(device)
    wandb.watch(model)  # wandb로 모델 가중치와 그래디언트 모니터링 시작

    # Adam 옵티마이저 설정
    optimizer = optim.Adam(
        model.parameters(),
        lr=wandb.config.learning_rate,      # 학습률
        weight_decay=wandb.config.weight_decay  # L2 정규화
    )

    # 학습 객체 생성
    regression_trainer = RegressionTrainer(
        project_name=project_name,           # 프로젝트 이름
        model=model,                         # 학습할 모델
        optimizer=optimizer,                 # 옵티마이저
        train_data_loader=train_data_loader,  # 학습 데이터 로더
        validation_data_loader=validation_data_loader,  # 검증 데이터 로더
        test_data_loader=None,              # 테스트 데이터 로더 (사용 안 함)
        run_time_str=run_time_str,          # 실행 식별자
        wandb=wandb,                        # wandb 객체
        device=device,                      # 학습 디바이스
        checkpoint_file_path=CHECKPOINT_FILE_PATH  # 모델 저장 경로
    )
    
    # 학습 실행
    regression_trainer.train_loop()

    # wandb 실행 종료
    wandb.finish()

In [33]:
if __name__ == "__main__":
    import sys
    if 'ipykernel' in sys.modules:  # Jupyter Notebook에서 실행 중인지 확인
        # Jupyter에서 실행할 때는 기본값 사용
        args = Args()  # Args 클래스의 인스턴스 생성
    else:
        # 일반 Python 스크립트로 실행할 때는 argparse 사용
        parser = get_parser()
        args = parser.parse_args()
    
    main(args)  # main 함수 호출은 if 문 내부에 있어야 함

{'epochs': 500, 'batch_size': 16, 'validation_intervals': 1, 'learning_rate': 0.0005, 'early_stop_patience': 50, 'early_stop_delta': 1e-06, 'weight_decay': 1e-05}
Training on device cuda:0.
[Epoch   1] T_loss: 0.62965, V_loss: 1.25612, Early stopping is stated! | T_time: 00:00:08, T_speed: 0.001
[Epoch   2] T_loss: 0.04792, V_loss: 1.90025, Early stopping counter: 1 out of 50 | T_time: 00:00:17, T_speed: 0.006
[Epoch   3] T_loss: 0.04721, V_loss: 0.73700, V_loss decreased (1.25612 --> 0.73700). Saving model... | T_time: 00:00:26, T_speed: 0.014
[Epoch   4] T_loss: 0.03728, V_loss: 1.70437, Early stopping counter: 1 out of 50 | T_time: 00:00:36, T_speed: 0.007
[Epoch   5] T_loss: 0.04079, V_loss: 0.77563, Early stopping counter: 2 out of 50 | T_time: 00:00:45, T_speed: 0.009
[Epoch   6] T_loss: 0.03659, V_loss: 0.94905, Early stopping counter: 3 out of 50 | T_time: 00:00:54, T_speed: 0.013
[Epoch   7] T_loss: 0.03573, V_loss: 2.12736, Early stopping counter: 4 out of 50 | T_time: 00:01:

Epoch,▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇██
Training loss,▅█▇▅▇▅▅▇▄▅▅▃▃▄▃▃▃▂▃▂▂▃▂▂▂▂▂▂▁▂▁▂▂▁▂▂▂▂▂▃
Training speed (epochs/sec.),▁▁▁▁█▁▁▁▁▁▁▁▆▁▁▂▃▃▂▁▁▁▂▆▁▂▁▄▂▄▃▃▃▁▂▂▂▂▂▄
Validation loss,▅█▄▃▅▃▂▄▂▆▄▂▄▄▅▂▄▃▃▃▃▁▂▃▂▅▃▃▂▂▂▂▂▄▂▄▄▃▄▄
Epoch,153
Training loss,0.02202
Training speed (epochs/sec.),0.68604
Validation loss,0.61033
